In [2]:
# db: simple read test
from utils.db_utils import get_df, get_data, connect_db, DB, insert_data, generate_schema

dump_folder = '/root/Projects/SG-VLN-axi/dump/5_04_hm3d_baseline_axi'
# dump_folder = './dump/baseline_apr20_sample400'
output_folder = f'{dump_folder}/objectnav-dino'

# list db size
! ls -lh $output_folder

total 51K
drwxrwxrwx 44 root root   45 May  5 05:00 episodes
drwxrwxrwx  2 root root   32 May  5 04:59 episodes_video
drwxrwxrwx  2 root root    4 May  4 22:13 logs
-rwxrwxrwx  1 root root  35K May  4 21:54 monitor.html
-rw-r--r--  1 root root 780K May  5 05:00 result.db


In [ ]:
%load_ext autoreload
%autoreload 2
from utils.db_utils import get_df, get_data, connect_db, DB, insert_data, generate_schema
with DB(f'/root/Projects/SG-VLN-axi/dump/5_04_hm3d_baseline_axi/objectnav-dino/result.db') as con:
    data = {'aaa': 1, 'bbb': 2}
    schema = generate_schema(data)
    con.create_table('step_data', schema=schema, overwrite=True)
    table = con.table('step_data')
    print('column ',table.columns)
    con.raw_sql("SET checkpoint_threshold = '100G';")
    con.raw_sql("BEGIN TRANSACTION;")
    insert_data(con, 'step_data', data)
    con.raw_sql("COMMIT;")
    con.raw_sql("CHECKPOINT;")
    con.raw_sql("BEGIN TRANSACTION;")
    data = {'aaa': 1, 'ccc': '123'}
    insert_data(con, 'step_data', data)
    data = {'aaa': 123, 'ddd': 2}
    insert_data(con, 'step_data', data)
    con.raw_sql("COMMIT;")
    con.raw_sql("CHECKPOINT;")
    print(con.table('step_data').columns)
print(get_data(f'/root/Projects/SG-VLN-axi/dump/5_04_hm3d_baseline_axi/objectnav-dino/result.db', 'step_data'))

In [ ]:
schema

In [3]:
# load results
results = get_df(f'{output_folder}/result.db', 'result')
print(f'Loaded {len(results)} results')
print(f'Current success rate: {results.tail(1)["success"].values[0]/len(results):.2%}')
print(f'Current SPL: {results["spl"].mean():.2f}')

Loaded 30 results
Current success rate: 76.67%
Current SPL: 0.35


In [4]:
# episode infos
steps_df = get_df(f'{output_folder}/step_data.db', 'step_data', select=['step', 'timestamp', 'process_label', 'episode_label'])
steps_df.groupby('episode_label').agg({'step': 'max', 'timestamp': 'max'})

IOException: IO Error: Cannot open database "/root/Projects/SG-VLN-axi/dump/5_04_hm3d_baseline_axi/objectnav-dino/step_data.db" in read-only mode: database does not exist

In [ ]:
# access db
with DB(f'{output_folder}/step_data.db') as con:
    table = con.table('step_data')
    print(table)

In [ ]:
# episode infos
steps = get_data(
    f'{output_folder}/step_data.db',
    'step_data',
    filter=lambda x: (x['step']%5==0) & (x['episode_label']=='cvZr5TUy5C5_25'),
    select=['step', 'timestamp', 'process_label', 'global_scene_graph', 'global_bev_rgb_map', 'global_bev_rgb_map_shape']
)
for i,step in enumerate(steps):
    print(i,step['step'])
print(steps[0]['global_scene_graph'])

TableNotFound: step_data

In [ ]:
import matplotlib.pyplot as plt
print(steps[3]['global_bev_rgb_map'].shape)
plt.imshow(steps[13]['global_bev_rgb_map'][...,::-1]/255.)

In [ ]:
import pickle
# episode infos
steps_df = get_df(
    f'{output_folder}/step_data.db',
    'step_data',
    filter=lambda x: (x['step']==35) & (x['episode_label']=='6s7QHgap2fW_89'),
    select=['step', 'timestamp', 'episode_label', 'objects_pickle']
)
steps_df
objects = pickle.loads(steps_df['objects_pickle'].values[0])
objects

In [ ]:
steps_df

In [ ]:
# TODO: see why these fails
# results = get_df(f'{output_folder}/result.db', 'result', filter=lambda x: (x['habitat_success']==0) & (x['distance_to_goal']<0.2))
results = get_df(f'{output_folder}/result.db', 'result')
results

In [ ]:
import duckdb
dump_folder = './dump/offline_analysis_sample100_apr23_copy/'
output_folder = f'{dump_folder}/objectnav-dino'
con = duckdb.connect(f'{output_folder}/step_data.db')

# data = con.execute("PRAGMA table_info('step_data')").fetchall()
# for row in data:
#     print(row)

data = con.execute("PRAGMA storage_info('step_data')").fetchall()
column_size = {}
table_size = 0
for row in data:
    # print(row)
    row_group_id = row[0]
    column_name = row[1]
    start = row[6]
    count = row[7]
    compression = row[8]
    if column_name not in column_size:
        column_size[column_name] = 0
    column_size[column_name] += count
    table_size += count
    print(f"row_group_id: {row_group_id}, Column: {column_name}, Start: {start}, Count: {count}, Compression: {compression}")
# for column_name, count in column_size.items():
#     print(f"Column: {column_name}, Count: {count}")

In [ ]:
for index, step in enumerate(steps):
    if step['global_scene_graph']:
        print(index, step['step'], step['time'], step['process_label'], step['episode_label'])
        print(step['global_scene_graph'])

In [ ]:
df.groupby('episode_label').size().reset_index(name='count')

test = con.table('test')
print(test.select(['step', 'success', 'info', 'time']).head(10).execute())
print('max score:', test["info"]["score"].unwrap_as("float32").max().execute())
print('mean score:', test["info"]["score"].unwrap_as("float32").mean().execute())
print('mean step:', test["step"].max().execute())
print('success rate:', test["success"].mean().execute())


In [ ]:
# test insert (with db utils)
%load_ext autoreload
%autoreload 2
import numpy as np
import ibis
import ibis.selectors as s
import os
import time
import datetime
from utils.db_utils import *
ibis.options.interactive = True
output_path = 'dump/scene_graph_testing/objectnav-dino'
db_path = f'{output_path}/test_insert.db'
os.remove(db_path) if os.path.exists(db_path) else None

with DB(db_path) as con:
    data = {'test': np.random.rand(1024*1024), 'label': 123}
    schema = generate_schema(data)
    print(schema)
    con.create_table('test', schema=schema, overwrite=True)
    total_start_time = time.time()
    con.raw_sql("BEGIN TRANSACTION;")
    for i in range(100):
        start_time = time.time()
        insert_data(con, 'test', data)
        print(f'insert time: {time.time()-start_time:.2f}s')
    con.raw_sql("COMMIT;")
    print(f'total time: {time.time()-total_start_time:.2f}s')
    end_time = time.time()
print(f'insert time: {time.time()-end_time:.2f}s')

In [ ]:
# test insert/commit/checkpoint with multiple dbs
%load_ext autoreload
%autoreload 2
import numpy as np
import ibis
import ibis.selectors as s
import os
import io
import time
import datetime

from utils.db_utils import *
ibis.options.interactive = True
output_path = 'dump/scene_graph_testing/objectnav-dino'
db_path = f'{output_path}/test_insert.db'
os.remove(db_path) if os.path.exists(db_path) else None

def generate_data():
    data = {'test': np.random.rand(1024,1024), 'label': 123}
    return data

# for k in range(3):
k=0
print(f'writing to {db_path}.{k}')
with DB(db_path+f'.{k}') as con:
    # con.raw_sql("PRAGMA disabled_compression_methods = 'alp,rle';")
    # con.raw_sql("PRAGMA force_compression = 'bitpacking';")
    data = generate_data()
    print(len(data['test'])/1024/1024)

    schema = generate_schema(data)
    print(schema)
    con.raw_sql("SET checkpoint_threshold = '100G';")
    con.create_table('test'+f'_{k}', schema=schema, overwrite=True)
    total_start_time = time.time()
    for j in range(3):
        for i in range(3):
            con.raw_sql("BEGIN TRANSACTION;")
            for i in range(3):
                start_time = time.time()
                insert_data(con, 'test'+f'_{k}', generate_data())
                # print(f'insert time: {time.time()-start_time:.2f}s')
            start_time = time.time()
            con.raw_sql("COMMIT;")
            print(f'commit time: {time.time()-start_time:.2f}s')
        start_time = time.time()
        con.raw_sql("CHECKPOINT;")
        print(f'checkpoint time: {time.time()-start_time:.2f}s')
    print(f'total time: {time.time()-total_start_time:.2f}s')
    end_time = time.time()
!ls -lh $output_path

print(f'insert time: {time.time()-end_time:.2f}s')

In [ ]:
# test reading from multiple dbs
k=0
with DB(f'{db_path}.0') as con:
    con.attach(f'{db_path}.2', 'main1')
    print(con.list_catalogs())
    print(con.list_tables())
    print(con.list_databases(catalog='main1'))
    print(con.list_tables(database='main1.main'))
    print(con.table('main1.main.test_0').execute().head(10))
    # table = con.table('test')
    # print(table.execute().groupby('label').agg({'label': 'count'}))

In [ ]:
?con

In [ ]:
# test insert (with db utils)
%load_ext autoreload
%autoreload 2
import numpy as np
import ibis
import ibis.selectors as s
import os
import time
import pickle
import datetime
from utils.db_utils import to_ibis_type, generate_schema, insert_data, generate_test_data, to_ibis_value, connect_db, analyze_data_size
ibis.options.interactive = True
output_path = 'dump/scene_graph_testing/objectnav-dino'
db_path = f'{output_path}/test_insert.db'


# save data as bytes
os.remove(db_path) if os.path.exists(db_path) else None
with DB(db_path) as con:
    data = [{'test': pickle.dumps(np.random.rand(1024*1024*10))} for i in range(10)]
    schema = generate_schema(data)
    print(schema)
    con.create_table('test', schema=schema, overwrite=True)
    total_start_time = time.time()
    con.raw_sql("BEGIN TRANSACTION;")
    insert_data(con, 'test', data)
    con.raw_sql("COMMIT;")
    print(f'total time: {time.time()-total_start_time:.2f}s')

# save data with pickle and time it
with open(f'{output_path}/test.pkl', 'wb') as f:
    start_time = time.time()
    pickle.dump(data, f)
    print(f'pickle dump time: {time.time()-start_time:.2f}s')

# df = con.table('test').to_pandas()
# df.head()

In [ ]:
from utils.db_utils import generate_test_data, analyze_data_size
analyze_data_size(generate_test_data(), threshold=0.00001)

In [ ]:
# ibis tutorial
import ibis
import ibis.selectors as s
ibis.options.interactive = True
output_path = '/dump/scene_graph_testing/objectnav-dino'
con = ibis.connect(f"duckdb:/{output_path}/penguins.db")
con.create_table(
    "penguins", ibis.examples.penguins.fetch().to_pyarrow(), overwrite=True
)
penguins = con.table("penguins")
penguins.head(10)
print('test1: ', penguins.filter((penguins.island == "Torgersen") & (penguins.species == "Adelie")))
print('test2: ', penguins.select("species", "island", "year"))

print('test3: ', penguins.mutate(
    bill_length_cm=penguins.bill_length_mm / 10,
    continent=ibis.literal("Antarctica")
).select("bill_length_cm").head(10))
print('test4: ', penguins.select("island", s.numeric()))
print('test5: ', penguins.order_by(penguins.flipper_length_mm.desc()).select(
    "species", "island", "flipper_length_mm"
).head(10))
print('test6: ', penguins.flipper_length_mm.mean())
print('test7: ', penguins.group_by(["species", "island"]).aggregate(
    [penguins.bill_length_mm.mean(), penguins.flipper_length_mm.max()]
))

In [ ]:
# test db: compare duckdb and numpy
import numpy as np
import ibis
import ibis.selectors as s
import os
import time
ibis.options.interactive = True
output_path = 'dump/scene_graph_testing/objectnav-dino'

# test1
db_path = f'{output_path}/test1.ddb'
os.remove(db_path) if os.path.exists(db_path) else None
con = ibis.connect(f"duckdb://{db_path}")
schema = ibis.schema(
    {
        'step': "int",
        'successs': int,
        'feat': "array<float32>",
    }
)
con.create_table('test', schema=schema, overwrite=True)
array_list = []
for i in range(10):
    test_array = np.ones(480*640*3).astype(np.float32)
    array_list.append(test_array)
time_start = time.time()
con.insert('test', {'step':[800]*len(array_list), 'successs':[1]*len(array_list), 'feat': array_list})
print(f'Time taken for duckdb: {time.time() - time_start} seconds')
con.disconnect()
start_time = time.time()
np.save(f'./{output_path}/test1.npy', array_list)
print(f'Time taken for numpy: {time.time() - start_time} seconds')
time.sleep(1)

# test2
db_path = f'{output_path}/test2.ddb'
os.remove(db_path) if os.path.exists(db_path) else None
con = ibis.connect(f"duckdb://{db_path}")
schema = ibis.schema(
    {
        'step': "int",
        'successs': int,
        'feat': "binary",
    }
)
con.create_table('test', schema=schema, overwrite=True)
array_list = []
for i in range(10):
    test_array = np.random.rand(480, 640, 3).astype(np.float32)
    array_list.append(test_array.tobytes())
time_start = time.time()
con.insert('test', {'step':[800]*len(array_list), 'successs':[1]*len(array_list), 'feat': array_list})
print(f'Time taken for duckdb: {time.time() - time_start} seconds')
con.disconnect()
start_time = time.time()
np.save(f'./{output_path}/test2.npy', array_list)
print(f'Time taken for numpy: {time.time() - start_time} seconds')
! ls -lh {output_path}/